In [1]:
import pandas as pd
import requests
import json
import math
import logging
from datetime import datetime
import pytz

In [2]:
pd.options.mode.chained_assignment = None  # default='warn'

In [5]:
datetime_utc = datetime.now(pytz.timezone('UTC'))
datetime_pst = datetime_utc.astimezone(pytz.timezone('US/Pacific'))
date_pst = datetime_pst.strftime('%Y-%m-%d')
timestamp_pst = datetime_pst.strftime('%Y-%m-%d %H:%M:%S')
year = datetime_pst.year

In [3]:
forms = {'460A':'0','460B1':'12','460C':'1','460D':'5','497P1':'20','496':'19'}
payload = {'format':'json'}
output_path = './././data/temp/netfile.csv'

In [22]:
datesSheet = 'http://seshat.datasd.org.s3.amazonaws.com/campaign_fin/reporting_dates.csv'
deadlines = pd.read_csv(datesSheet, delimiter=',')
currentDeadlines = deadlines[(deadlines['year'] == year) & (deadlines['form'] == '460')]
currentDeadlines['deadline'] = pd.to_datetime(currentDeadlines['deadline'])
currentDeadlines['period_to'] = pd.to_datetime(currentDeadlines['period_to'])
currentDeadlines.sort_values(['deadline'],inplace=True)
currentDeadlines['deadline_next'] = currentDeadlines['deadline'].shift(-1)
lastDeadline = currentDeadlines[(timestamp_pst > currentDeadlines['deadline']) & (timestamp_pst <= currentDeadlines['deadline_next'])].iloc[0]['period_to']

In [23]:
print(lastDeadline)

2018-06-30 00:00:00


In [4]:
outputDF = pd.DataFrame()
dateResponse = 'dateTK'
prod_columns = ['form',
  'schedule',
  'schedule_description',
  'recipient_id',
  'recipient_name',
  'report_period_from',
  'report_period_to',
  'contributor_code',
  'contributor_last',
  'contributor_first',
  'contributor_city',
  'contributor_state',
  'contributor_zip',
  'contributor_emp',
  'contributor_occ',
  'contribution_date',
  'contribution_amount',
  'contribution_annual',
  'contribution_desc',
  'contributor_id',
  'intermediary_last',
  'intermediary_first',
  'intermediary_city',
  'intermediary_state',
  'intermediary_zip',
  'intermediary_emp',
  'intermediary_occ',
  'filing_id',
  'report_year']

In [8]:
countRequest = requests.post('https://netfile.com:443/Connect2/api/public/campaign/export/cal201/transaction/year', params=payload,
                    data = {'Aid':'CSD',
                            'Year':2018,
                            'CurrentPageIndex':0,
                            'PageSize':'1',
                            'TransactionType':forms['460A'],
                            'ShowSuperceded':'false'})
if countRequest.status_code == 200:
    print("Success")
else:
    print("bad")

Success


In [6]:
countRequest = requests.post('https://netfile.com:443/Connect2/api/public/campaign/export/cal201/transaction/year', params=payload,
                    data = {'Aid':'CSD',
                            'Year':year,
                            'CurrentPageIndex':0,
                            'PageSize':'1',
                            'TransactionType':forms['460A'],
                            'ShowSuperceded':'false'})
formTransactions = countRequest.json()['totalMatchingCount']

if formTransactions < 1000:
    requestLoops = 1
else:
    requestLoops = math.ceil(formTransactions/1000)
      
transactionsList = []

for i in range(requestLoops):
    page = str(i)
    transactionsRequest = requests.post('https://netfile.com:443/Connect2/api/public/campaign/export/cal201/transaction/year', params=payload,
                    data = {'Aid':'CSD',
                            'Year':year,
                            'CurrentPageIndex':page,
                            'PageSize':'1000',
                            'TransactionType':forms['460A'],
                            'ShowSuperceded':'false'})
    allTransactions = transactionsRequest.json()['results']
      
    for t in allTransactions:
          # Create row
        transactionsList.append(['460', #form
                                       t['form_Type'], #schedule
                                       'Monetary contributions', #schedule_description
                                       t['filerStateId'], #recipient_id
                                       t['filerName'], #recipient_name
                                       t['filingStartDate'], #'report_period_from'
                                       t['filingEndDate'], #'report_period_to'
                                       t['entity_Cd'], #'contributor_code'
                                       t['tran_NamL'], #'contributor_last'
                                       t['tran_NamF'], #'contributor_first'
                                       t['tran_City'], #'contributor_city'
                                       t['tran_ST'], #'contributor_state'
                                       t['tran_Zip4'], #'contributor_zip'
                                       t['tran_Emp'], #'contributor_emp'
                                       t['tran_Occ'], #'contributor_occ'
                                       t['tran_Date'], #'contribution_date'
                                       t['tran_Amt1'], #'contribution_amount'
                                       t['tran_Amt2'], #'contribution_annual'
                                       t['tran_Dscr'], #'contribution_desc'
                                       t['cmte_Id'], #'contributor_id'
                                       t['intr_NamL'], #'intermediary_last'
                                       t['intr_NamF'], #'intermediary_first'
                                       t['intr_City'], #'intermediary_city'
                                       t['intr_ST'], #'intermediary_state'
                                       t['intr_Zip4'], #'intermediary_zip'
                                       t['intr_Emp'], #'intermediary_emp'
                                       t['intr_Occ'], #'intermediary_occ'
                                       t['filingId'],
                                         year
                                       ])


campaignTransactions = pd.DataFrame(transactionsList,columns=prod_columns)        
outputDF = pd.concat([outputDF, campaignTransactions], ignore_index=True)

In [9]:
countRequest = requests.post('https://netfile.com:443/Connect2/api/public/campaign/export/cal201/transaction/year', params=payload,
                    data = {'Aid':'CSD',
                            'Year':year,
                            'CurrentPageIndex':'0',
                            'PageSize':'1',
                            'TransactionType':forms['460B1'],
                            'ShowSuperceded':'false'})
formTransactions = countRequest.json()['totalMatchingCount']

if formTransactions < 1000:
    requestLoops = 1
else:
    requestLoops = math.ceil(formTransactions/1000)
      
transactionsList = []

for i in range(requestLoops):
    page = str(i)
    transactionsRequest = requests.post('https://netfile.com:443/Connect2/api/public/campaign/export/cal201/transaction/year', params=payload,
                    data = {'Aid':'CSD',
                            'Year':year,
                            'CurrentPageIndex':page,
                            'PageSize':'1000',
                            'TransactionType':forms['460B1'],
                            'ShowSuperceded':'false'})
    allTransactions = transactionsRequest.json()['results']
      
    for t in allTransactions:
          # Create row
        transactionsList.append(['460', #form
                                       t['form_Type'], #schedule
                                       'Loans', #schedule_description
                                       t['filerStateId'], #recipient_id
                                       t['filerName'], #recipient_name
                                       t['filingStartDate'], #'report_period_from'
                                       t['filingEndDate'], #'report_period_to'
                                       t['entity_Cd'], #'contributor_code'
                                       t['tran_NamL'], #'contributor_last'
                                       t['tran_NamF'], #'contributor_first'
                                       t['tran_City'], #'contributor_city'
                                       t['tran_ST'], #'contributor_state'
                                       t['tran_Zip4'], #'contributor_zip'
                                       t['tran_Emp'], #'contributor_emp'
                                       t['tran_Occ'], #'contributor_occ'
                                       t['tran_Date'], #'contribution_date'
                                       t['tran_Amt1'], #'contribution_amount'
                                       t['tran_Amt2'], #'contribution_annual'
                                       t['tran_Dscr'], #'contribution_desc'
                                       t['cmte_Id'], #'contributor_id'
                                       t['intr_NamL'], #'intermediary_last'
                                       t['intr_NamF'], #'intermediary_first'
                                       t['intr_City'], #'intermediary_city'
                                       t['intr_ST'], #'intermediary_state'
                                       t['intr_Zip4'], #'intermediary_zip'
                                       t['intr_Emp'], #'intermediary_emp'
                                       t['intr_Occ'], #'intermediary_occ'
                                       t['filingId'],
                                         year
                                       ])


campaignTransactions = pd.DataFrame(transactionsList,columns=prod_columns)        
outputDF = pd.concat([outputDF, campaignTransactions], ignore_index=True)

In [10]:
countRequest = requests.post('https://netfile.com:443/Connect2/api/public/campaign/export/cal201/transaction/year', params=payload,
                    data = {'Aid':'CSD',
                            'Year':year,
                            'CurrentPageIndex':'0',
                            'PageSize':'1000',
                            'TransactionType':forms['460C'],
                            'ShowSuperceded':'false'})
formTransactions = countRequest.json()['totalMatchingCount']

if formTransactions < 1000:
    requestLoops = 1
else:
    requestLoops = math.ceil(formTransactions/1000)
      
transactionsList = []

for i in range(requestLoops):
    page = str(i)
    transactionsRequest = requests.post('https://netfile.com:443/Connect2/api/public/campaign/export/cal201/transaction/year', params=payload,
                    data = {'Aid':'CSD',
                            'Year':year,
                            'CurrentPageIndex':page,
                            'PageSize':'1000',
                            'TransactionType':forms['460C'],
                            'ShowSuperceded':'false'})
    allTransactions = transactionsRequest.json()['results']
      
    for t in allTransactions:
          # Create row
        transactionsList.append(['460', #form
                                       t['form_Type'], #schedule
                                       'Non monetary contributions', #schedule_description
                                       t['filerStateId'], #recipient_id
                                       t['filerName'], #recipient_name
                                       t['filingStartDate'], #'report_period_from'
                                       t['filingEndDate'], #'report_period_to'
                                       t['entity_Cd'], #'contributor_code'
                                       t['tran_NamL'], #'contributor_last'
                                       t['tran_NamF'], #'contributor_first'
                                       t['tran_City'], #'contributor_city'
                                       t['tran_ST'], #'contributor_state'
                                       t['tran_Zip4'], #'contributor_zip'
                                       t['tran_Emp'], #'contributor_emp'
                                       t['tran_Occ'], #'contributor_occ'
                                       t['tran_Date'], #'contribution_date'
                                       t['tran_Amt1'], #'contribution_amount'
                                       t['tran_Amt2'], #'contribution_annual'
                                       t['tran_Dscr'], #'contribution_desc'
                                       t['cmte_Id'], #'contributor_id'
                                       t['intr_NamL'], #'intermediary_last'
                                       t['intr_NamF'], #'intermediary_first'
                                       t['intr_City'], #'intermediary_city'
                                       t['intr_ST'], #'intermediary_state'
                                       t['intr_Zip4'], #'intermediary_zip'
                                       t['intr_Emp'], #'intermediary_emp'
                                       t['intr_Occ'], #'intermediary_occ'
                                       t['filingId'],
                                         year
                                       ])


campaignTransactions = pd.DataFrame(transactionsList,columns=prod_columns)        
outputDF = pd.concat([outputDF, campaignTransactions], ignore_index=True)

In [11]:
countRequest = requests.post('https://netfile.com:443/Connect2/api/public/campaign/export/cal201/transaction/year', params=payload,
                    data = {'Aid':'CSD',
                            'Year':year,
                            'CurrentPageIndex':'0',
                            'PageSize':'1',
                            'TransactionType':forms['460D'],
                            'ShowSuperceded':'false'})
formTransactions = countRequest.json()['totalMatchingCount']

if formTransactions < 1000:
    requestLoops = 1
else:
    requestLoops = math.ceil(formTransactions/1000)
      
transactionsList = []

for i in range(requestLoops):
    page = str(i)
    transactionsRequest = requests.post('https://netfile.com:443/Connect2/api/public/campaign/export/cal201/transaction/year', params=payload,
                    data = {'Aid':'CSD',
                            'Year':year,
                            'CurrentPageIndex':page,
                            'PageSize':'1000',
                            'TransactionType':forms['460D'],
                            'ShowSuperceded':'false'})
    allTransactions = transactionsRequest.json()['results']
      
    for t in allTransactions:
        if t['sup_Opp_Cd'] == 'S':
            candName = t['cand_NamF']+' '+t['cand_NamL']
            balName = t['bal_Name']
            otherNameF = t['tran_NamF']
            otherNameL = t['tran_NamL']
            if balName != '':
                fullName = balName
            elif candName != ' ':
                fullName = candName.strip()
            else:
                fullName = otherNameL
        
              # Create row
            transactionsList.append(['460', #form
                                       t['form_Type'], #schedule
                                       'Independent expenditures in support', #schedule_description
                                       t['cmte_Id'], #recipient_id
                                       fullName, #recipient_name
                                       t['filingStartDate'], #'report_period_from'
                                       t['filingEndDate'], #'report_period_to'
                                       ' ', #'contributor_code'
                                       t['filerName'], #'contributor_last'
                                       ' ', #'contributor_first'
                                       ' ', #'contributor_city'
                                       ' ', #'contributor_state'
                                       ' ', #'contributor_zip'
                                       ' ', #'contributor_emp'
                                       ' ', #'contributor_occ'
                                       t['tran_Date'], #'contribution_date'
                                       t['tran_Amt1'], #'contribution_amount'
                                       t['tran_Amt2'], #'contribution_annual'
                                       t['tran_Dscr'], #'contribution_desc'
                                       t['filerStateId'], #'contributor_id'
                                       t['intr_NamL'], #'intermediary_last'
                                       t['intr_NamF'], #'intermediary_first'
                                       t['intr_City'], #'intermediary_city'
                                       t['intr_ST'], #'intermediary_state'
                                       t['intr_Zip4'], #'intermediary_zip'
                                       t['intr_Emp'], #'intermediary_emp'
                                       t['intr_Occ'], #'intermediary_occ'
                                       t['filingId'],
                                       year
                                       ])


campaignTransactions = pd.DataFrame(transactionsList,columns=prod_columns)        
outputDF = pd.concat([outputDF, campaignTransactions], ignore_index=True)

In [12]:
countRequest = requests.post('https://netfile.com:443/Connect2/api/public/campaign/export/cal201/summary/year', params=payload,
                    data = {'Aid':'CSD',
                            'Year':year,
                            'CurrentPageIndex':'0',
                            'PageSize':'1',
                            'ShowSuperceded':'false'})
formTransactions = countRequest.json()['totalMatchingCount']

if formTransactions < 1000:
    requestLoops = 1
else:
    requestLoops = math.ceil(formTransactions/1000)
      
transactionsList = []
filingReferenceList = []

for i in range(requestLoops):
          
    page = str(i)
    transactionsRequest = requests.post('https://netfile.com:443/Connect2/api/public/campaign/export/cal201/summary/year', params=payload,
                    data = {'Aid':'CSD',
                            'Year':year,
                            'CurrentPageIndex':page,
                            'PageSize':'1000',
                            'ShowSuperceded':'false'})
    allTransactions = transactionsRequest.json()['results']
    
    for t in allTransactions:
        if t['form_Type'] == 'A' or t['form_Type'] == 'C':
            if t['line_Item'] == '2':
                if t['form_Type'] == 'A':
                    description = 'Unitemized monetary contributions less than $100'
                    schedule = 'SMRY A'
                    filingReferenceList.append([t['filerStateId'],pd.to_datetime(t['filingEndDate'])])
                elif t['form_Type'] == 'C':
                    description = 'Unitemized nonmonetary contributions less than $100'
                    schedule = 'SMRY C'
                  # Create row
                transactionsList.append(['460', #form
                                       schedule, #schedule
                                       description, #schedule_description
                                       t['filerStateId'], #recipient_id
                                       t['filerName'], #recipient_name
                                       t['filingStartDate'], #'report_period_from'
                                       t['filingEndDate'], #'report_period_to'
                                       ' ', #'contributor_code'
                                       ' ', #'contributor_last'
                                       ' ', #'contributor_first'
                                       ' ', #'contributor_city'
                                       ' ', #'contributor_state'
                                       ' ', #'contributor_zip'
                                       ' ', #'contributor_emp'
                                       ' ', #'contributor_occ'
                                       ' ', #'contribution_date'
                                       t['amount_A'], #'contribution_amount'
                                       t['amount_B'], #'contribution_annual'
                                       ' ', #'contribution_desc'
                                       ' ', #'contributor_id'
                                       ' ', #'intermediary_last'
                                       ' ', #'intermediary_first'
                                       ' ', #'intermediary_city'
                                       ' ', #'intermediary_state'
                                       ' ', #'intermediary_zip'
                                       ' ', #'intermediary_emp'
                                       ' ', #'intermediary_occ'
                                       t['filingId'],
                                        year
                                       ])


campaignTransactions = pd.DataFrame(transactionsList,columns=prod_columns)        
outputDF = pd.concat([outputDF, campaignTransactions], ignore_index=True)

In [16]:
date_checks = pd.DataFrame(data=filingReferenceList,columns=['id','date'])
date_checks_max = date_checks.groupby(['id'])['date'].max()
date_checks_max = date_checks_max.to_frame()
date_checks_max.reset_index(inplace=True)

In [24]:
date_checks_max

,id,date
0,1234153,2017-06-30 07:00:00
1,1255438,2017-06-30 07:00:00
2,1258295,2017-06-30 07:00:00
3,1280768,2017-09-30 07:00:00
4,1326869,2017-06-30 07:00:00
5,1335733,2017-06-30 07:00:00
6,1337894,2017-06-30 07:00:00
7,1363108,2017-06-30 07:00:00
8,1366044,2017-06-30 07:00:00
9,1366059,2017-06-30 07:00:00


In [20]:
countRequest = requests.post('https://netfile.com:443/Connect2/api/public/campaign/export/cal201/transaction/year', params=payload,
                    data = {'Aid':'CSD',
                            'Year':year,
                            'CurrentPageIndex':0,
                            'PageSize':'1',
                            'TransactionType':forms['497P1'],
                            'ShowSuperceded':'false'})
formTransactions = countRequest.json()['totalMatchingCount']

if formTransactions < 1000:
    requestLoops = 1
else:
    requestLoops = math.ceil(formTransactions/1000)
      
transactionsList = []

for i in range(requestLoops):
    page = str(i)
    transactionsRequest = requests.post('https://netfile.com:443/Connect2/api/public/campaign/export/cal201/transaction/year', params=payload,
                    data = {'Aid':'CSD',
                            'Year':year,
                            'CurrentPageIndex':page,
                            'PageSize':'1000',
                            'TransactionType':forms['497P1'],
                            'ShowSuperceded':'false'})
    allTransactions = transactionsRequest.json()['results']
      
    for t in allTransactions:
          # Create row
        transaction_date = pd.to_datetime(t['tran_Date'])
        for i, row in date_checks_max.iterrows():
            if row[0] == t['filerStateId']:
                if transaction_date > row[1]:
                    transactionsList.append(['497', #form
                                       t['form_Type'], #schedule
                                       '24-hr contribution report', #schedule_description
                                       t['filerStateId'], #recipient_id
                                       t['filerName'], #recipient_name
                                       t['filingStartDate'], #'report_period_from'
                                       t['filingEndDate'], #'report_period_to'
                                       t['entity_Cd'], #'contributor_code'
                                       t['tran_NamL'], #'contributor_last'
                                       t['tran_NamF'], #'contributor_first'
                                       t['tran_City'], #'contributor_city'
                                       t['tran_ST'], #'contributor_state'
                                       t['tran_Zip4'], #'contributor_zip'
                                       t['tran_Emp'], #'contributor_emp'
                                       t['tran_Occ'], #'contributor_occ'
                                       t['tran_Date'], #'contribution_date'
                                       t['tran_Amt1'], #'contribution_amount'
                                       t['tran_Amt2'], #'contribution_annual'
                                       t['tran_Dscr'], #'contribution_desc'
                                       t['cmte_Id'], #'contributor_id'
                                       t['intr_NamL'], #'intermediary_last'
                                       t['intr_NamF'], #'intermediary_first'
                                       t['intr_City'], #'intermediary_city'
                                       t['intr_ST'], #'intermediary_state'
                                       t['intr_Zip4'], #'intermediary_zip'
                                       t['intr_Emp'], #'intermediary_emp'
                                       t['intr_Occ'], #'intermediary_occ'
                                       t['filingId'],
                                         year
                                       ])


campaignTransactions = pd.DataFrame(transactionsList,columns=prod_columns)        
outputDF = pd.concat([outputDF, campaignTransactions], ignore_index=True)

In [22]:
countRequest = requests.post('https://netfile.com:443/Connect2/api/public/campaign/export/cal201/transaction/year', params=payload,
                    data = {'Aid':'CSD',
                            'Year':year,
                            'CurrentPageIndex':'0',
                            'PageSize':'1',
                            'TransactionType':forms['496'],
                            'ShowSuperceded':'false'})
formTransactions = countRequest.json()['totalMatchingCount']

if formTransactions < 1000:
    requestLoops = 1
else:
    requestLoops = math.ceil(formTransactions/1000)
      
transactionsList = []

for i in range(requestLoops):
    page = str(i)
    transactionsRequest = requests.post('https://netfile.com:443/Connect2/api/public/campaign/export/cal201/transaction/year', params=payload,
                    data = {'Aid':'CSD',
                            'Year':year,
                            'CurrentPageIndex':page,
                            'PageSize':'1000',
                            'TransactionType':forms['496'],
                            'ShowSuperceded':'false'})
    allTransactions = transactionsRequest.json()['results']
      
    for t in allTransactions:
        if t['sup_Opp_Cd'] == 'S':
            candName = t['cand_NamF']+' '+t['cand_NamL']
            balName = t['bal_Name']
            otherNameF = t['tran_NamF']
            otherNameL = t['tran_NamL']
            if balName != '':
                fullName = balName
            elif candName != ' ':
                fullName = candName.strip()
            else:
                fullName = otherNameL
        
              # Create row
            transaction_date = pd.to_datetime(t['tran_Date'])
            for i, row in date_checks_max.iterrows():
                if row[0] == t['filerStateId']:
                    if transaction_date > row[1]:
                        transactionsList.append(['460', #form
                                       t['form_Type'], #schedule
                                       'Independent expenditures in support', #schedule_description
                                       t['cmte_Id'], #recipient_id
                                       fullName, #recipient_name
                                       t['filingStartDate'], #'report_period_from'
                                       t['filingEndDate'], #'report_period_to'
                                       ' ', #'contributor_code'
                                       t['filerName'], #'contributor_last'
                                       ' ', #'contributor_first'
                                       ' ', #'contributor_city'
                                       ' ', #'contributor_state'
                                       ' ', #'contributor_zip'
                                       ' ', #'contributor_emp'
                                       ' ', #'contributor_occ'
                                       t['tran_Date'], #'contribution_date'
                                       t['tran_Amt1'], #'contribution_amount'
                                       t['tran_Amt2'], #'contribution_annual'
                                       t['tran_Dscr'], #'contribution_desc'
                                       t['filerStateId'], #'contributor_id'
                                       t['intr_NamL'], #'intermediary_last'
                                       t['intr_NamF'], #'intermediary_first'
                                       t['intr_City'], #'intermediary_city'
                                       t['intr_ST'], #'intermediary_state'
                                       t['intr_Zip4'], #'intermediary_zip'
                                       t['intr_Emp'], #'intermediary_emp'
                                       t['intr_Occ'], #'intermediary_occ'
                                       t['filingId'],
                                       year
                                       ])


campaignTransactions = pd.DataFrame(transactionsList,columns=prod_columns)        
outputDF = pd.concat([outputDF, campaignTransactions], ignore_index=True)

In [23]:
outputDF.to_csv(output_path, index=False)

In [ ]:
#Bring in recipients data to check for new recipients, send alert

In [7]:
recipients_path = 'http://seshat.datasd.org/campaign_fin/financial_support_recipients_datasd.csv'
recipients = pd.read_csv(recipients_path, delimiter=',')

recipients_committees = recipients[recipients["committee_id"].notnull()]
recipients_candidates = recipients[recipients["committee_id"].isnull()]
unique_committees = recipients_committees.committee_id.unique()
unique_candidates = recipients_candidates.recipient_name.unique()

committees_missing = outputDF[~outputDF['recipient_id'].isin(unique_committees)]
candidates_missing = outputDF[~outputDF['recipient_name'].isin(unique_candidates)]

In [8]:
committees_missing_unique = pd.Series(committees_missing['recipient_id'].unique())

In [9]:
unique_committees

array(['1298544', '1298534', '1235908', '1297980', '1298864', '1285407',
       '1270953', '1276908', '1299411', '1235921', '1282704', '1278616',
       '990466', '1301578', '1276918', '1234153', '1298855', '1302820',
       '831561/C00177766', '1278931', '1301052', '1254870', '1298853',
       '1302689', '1255438', '1298515', '1283366', '1297978', '1268914',
       '1285330', '741949', '1283641', '1260553', '76-1453', '1299425',
       '99-1330', '1301984', '1297982', '1283388', '1303774', '1237224',
       '1303810', '1304341', '1304578', '1305702', '1249289', '1310299',
       '741906', '744131', '1308115', '1304339', '1279034', '1305653',
       '1309024', '1314983', '1337381', '1318768', '1318781', '1319871',
       '1318770', '1307782', '1319545', '1280768', '1319122', '1318850',
       '1320421', '1322855', '1322891', '1309244', '1319024', '1319082',
       '1330949', '1241835', '810163', '1327333', '1323704', '902797',
       '1330359', '1326869', '1329881', '1331139', '1326995

In [10]:
committees_missing_unique

0     1399927
1     1394941
2     Pending
3     1397437
4     1400740
5     1394460
6     1394143
7     1397470
8     1397080
9     1399478
10    1398814
11    1395292
12    1400617
13    1396997
14    1399902
15    1399498
16    1396315
17    1394504
18    1395389
19    1395590
20    1397529
21    1402751
22    1394851
23    1405667
dtype: object

In [11]:
len(committees_missing_unique)

24